In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import csv
import time
import matplotlib.pyplot as plt

#Functions

def Get_Files (setfile,labelfile):
     
    with open(setfile, 'r') as file:
        Input_set = np.array(list(csv.reader(file, delimiter=',',quoting=csv.QUOTE_NONNUMERIC)))

    with open(labelfile, 'r') as labelfile:
        labellist = list(csv.reader(labelfile, delimiter=',',quoting=csv.QUOTE_NONNUMERIC))
        Output_set = np.array([item for sublist in labellist for item in sublist])
    
    desired_out = np.zeros([len(Output_set), 8], dtype = int)

    for i,v in enumerate(Output_set):
        desired_out[int(i), int(v-1),] = 1
    
    
    return Input_set, desired_out, Output_set


def activation_prompt(layer):
    """
    Chooses what activation function to use. 
        Activation Function 
        0 - Logistic Function
        1 - Tangent Function
        2 - ReLu
    Returns
    -------
        Activation Function
        Slope Parameter or Tanh Parameter a
        0 or Tanh Parameter b
    """
    
    print('Activation Function:')
    print('0: Logistic')
    print('1: Tangent Hyperbolic')
    print('2: ReLu')

    while True:
        try:
            question = int(input('Activation Function for '+ layer + ': '))
            break
        except:
            print("That's not a valid option!")

    if question == 0:
        print('Activation Function: Logistic')
        slope = float(input('Logistic Slope Parameter (Default is 2): ') or '2')
        return 0, slope, 0
    elif question == 1:
        
        print('Activation Function: Tangent Hyperbolic')
        tanh_a = float(input('Tanh a Parameter (Default is 1.716): ') or '1.716')
        tanh_b = float(input('Tanh b Parameter (Default is 2/3): ') or str(2/3))
        return 1, tanh_a, tanh_b
    elif question == 2:
        print('Activation Function: Leaky ReLu')
        slope1 = float(input('Input > 0 Slope Parameter (Default is 1): ') or '1')
        slope2 = float(input('Input < 0 Slope Parameter (Default is 0.01): ') or '0.01')
        return 2, slope1, slope2
    else:
        print('That\'s not an option!')
        return np.nan,np.nan,np.nan
    
def activation_function (H):
    """
    Calls the activation_prompt to initialize the activation functions per layer.
    """
    af = {}
    for i in range(H):
        l = str(i+1)
        layer = 'Hidden Layer ' + l
        p = activation_prompt(layer)
        af['af_'+ l]= p
    p = activation_prompt('Outer Layer')
    af['af_out'] = p
    return af
    
def weights (H,Input,Output):
    """Determine the Initial Weights and Delta Weights based on the number of nodes
        H: Number of Hidden Layer
        Input: array of numerics
            one instance with 354 features of the Input Set
        Output: array of numerics
            8 output classes
    """
    n = len(Input) +1
    m = len(Output)
    w = {}
    dw = {}
    nodes = int(input('Number of Nodes in 1st Hidden Layer (Default 100): ') or '100')
    for i in range(H):
        l = str(i+1)
        
        w['w_h'+l] = np.random.randn(nodes,n)*np.sqrt(2/n)
        dw['dw_h'+l] = np.zeros((nodes,n))
        n = nodes+1
        nodes = n+1
    w['w_out'] = np.random.randn(m,n)*np.sqrt(2/n)
    dw['dw_out'] = np.zeros((m,n))
    
    return w,dw


def output(p, v):
    """
    Calculates the Output value. 
    
    Parameters
    ----------
    p : tuple
        Activation Function:
            0 - Logistic Function
            1 - Tangent Function
            2 - ReLu
        a : Numeric
            For Logistic, enter the logistic slope parameter.
            For Tangent, enter the tanh parameter a
        b : Numeric
            For Tangent, enter the tanh parameter b.
 
    v : Numeric
        Internal activity, summation of the product of the inputs and the weights minus the bias.

    Returns
    -------
    Output

    """
    
    if p[0] == 0: #Logistic
        output = 1/(1+np.exp(-v*p[1]))
        return output
    elif p[0] == 1: #TanH
        output = p[1]*np.tanh(p[2]*v) 
        return output
    elif p[0] == 2:#Leaky ReLu
        output = np.maximum(p[2]*v,p[1]*v)
        return output
    else:
        return np.nan


def forward(Input, H, w, af):
    """ 
    Forward Propagation:
        Input: batch of input values
        H: Number of Hidden Layers
        w: Weights
        af: activation function per layer
    Returns
    -------    
        v: itnernal activity of batch
        y: output of batch
        
    """
    
    v = {}
    y = {}
    
    #Hidden Layer
    for i in range(H):
        l = str(i+1) 
        v['v_h'+l] = np.einsum('ij,kj->ik', Input, w['w_h'+l])
        y['y_h'+ l] = output(af['af_'+ l],v['v_h'+ l])
        Input = np.insert(y['y_h'+l], 0, 1, axis = 1)
    #Output Layer   
    v['v_out'] =np.einsum('ij,kj->ik', Input, w['w_out'])
    y['y_out'] = output(af['af_out'],v['v_out'])
    
    return v,y    
    
def error(d, y):
    """
    Calcualtes the error to be used for backpropagation
    d: desired output
    y: actual output
    --------
    e: Error
    """

    e = d-y
    return e

def reluDerivative(x,a,b):
    """
    Calculates the derivative of ReLu
    """
    x[x<=0] = b
    x[x>0] = a
    return x

def delta_function(layer,af,e,y,w,d):
    """
    Calculates the delta gradient based on the layer and activation function
        layer: Outer or Hidden Layer
            0: Outer Layer
            1: Hidden Layer
        af: activation function
            0: Logistic
            1: TanH
            2: Leaky Relu
        e: batch error
        y: batch actual output
        w: weights
        d: previous delta"""   
    
    if layer == 0: #Outer Layer
        if af[0] == 0: #Logistic
            delta = af[1]*e*y*(1-y)
            
        elif af[0] == 1: #Tanh
            delta = (af[2]/af[1])*e*(af[1]-y)*(af[1]+y)
        
        elif af[0] == 2: #Leaky ReLu
            delta = np.where(y > 0, y, y * af[1]) 
            
        else:
            delta = np.nan
    
    if layer == 1: #Hidden Layer
        if af[0] == 0: #Logistic
            delta =  af[1]*y*(1-y)*np.einsum('ij,jk->ik', d,  np.delete(w, (0), axis=1))             
         
        elif af[0] == 1: #Tanh
            delta = (af[2]/af[1])*(af[1]-y)*(af[1]+y)*np.einsum('ij,jk->ik', d,  np.delete(w, (0), axis=1))
        
        elif af[0] == 2: #Leaky ReLu
            delta = reluDerivative(y, af[1],af[2])*np.einsum('ij,jk->ik', d,  np.delete(w, (0), axis=1))
        
            
        else:
            delta = np.nan
    
    return delta


def gradient(H,e,af,y,w):
    """
    Calculates the delta gradient based on the activation function and the layer
        H: Number of Hidden Layers
        e: batch error
        af: activation function
        y: batch actual output
        w: weights
    """
    delta = {}
    
    #outer
    delta['d_out'] = delta_function(0,af['af_out'],e,y['y_out'],w['w_out'],0)
    
    l = H
    w_i = w['w_out']
    d_i = delta['d_out']
    #hidden layers
    for i in range(H):
        j = str(l)
        delta['d_h'+ j] = delta_function(1,af['af_' + j],e,y['y_h'+ j],w_i,d_i)
        l = l-1
        w_i = w['w_h'+j]
        d_i = delta['d_h'+ j]
    
    return delta

def delta_weights(H,Input,a,delta,y,batchsize):
    """
    Delta weights correction
        H: Number of Hidden Layers
        Input: batch Input
        a: learning rate
        delta: delta gradient
        y: batch actual output
        batchsize: number of Input sets in the batch
    """
    
    dw = {}
    for i in range(H):
        l = str(i+1)
        
        dw['dw_h'+l] = (a*np.einsum('ij,kl->lj', Input, delta['d_h'+l]))/np.square(len(Input))
        Input = np.insert(y['y_h'+l], 0, 1, axis = 1)
            
    dw['dw_out'] = (a*np.einsum('ij,kl->lj', Input, delta['d_out']))/np.square(len(Input))
   
    return dw


def new_weights (H,m,w1,dw_old,dw):
    """
    Weights update using the Generalized Delta Rule
        H: Number of Hidden Layers
        m: momentum
        w1: weights
        dw_old: old delta weights
        dw: new delta weights
    """
    
    w = {}
    for i in range(H):
        l = str(i+1)
        w['w_h'+l] = w1['w_h'+l] +  dw['dw_h'+l] + m*dw_old['dw_h'+l] 
    
    w['w_out'] = w1['w_out'] + dw['dw_out'] + m*dw_old['dw_out'] 
        
    return w

#%%
#Start Testing

bias = 1

print("Loading Training and Validation Files. Please Wait.")
#Initialize dataset
setfile = 'training_set.csv'
labelfile = 'training_label.csv'
InputSet,DesiredOutputSet,OutputSet = Get_Files(setfile,labelfile)

#Initialize validation dataset
setfile = 'validation_set.csv'
labelfile = 'validation_label.csv'
InputValidationSet,DesiredOutputValidationSet,OutputValidationSet = Get_Files(setfile,labelfile)
InputValidationSet = np.insert(InputValidationSet, 0, bias, axis=1)

#Shuffle Inputset
df = np.column_stack((InputSet,DesiredOutputSet))
df_shuffle = np.take(df,np.random.permutation(df.shape[0]),axis=0,out=df) 
outputsize = len(DesiredOutputSet[0,:])
inputsize = len(InputSet[0,:])

#Enter User Inputs
Network = "Network" + str(input('Network: '))
H = int(input('Number of Hidden Layers (Default 2): ') or '2')
batchsize = int(input('Batch Size (Default 8): ') or '8')
a = float(input('Learning Rate (Default 0.1): ') or '0.1')
ainit = a
m = float(input('Momentum (Default 0.9): ') or '0.9')
minit = m
af = activation_function(H)
w,dw_0 = weights(H,InputSet[0], DesiredOutputSet[0])
epoch = int(input('Number of Epoch (Default 5): ') or '5')

start_time = time.time()

#Start Training
epocherror = []
validation_error = []
misclassification = []
lt=[]
mplot=[]
print('Start of Learning: ---'+ str(start_time) +'---')
for k in range(epoch):
    epochstart_time = time.time()
    final_output = []
    sumerror = []
    lt.append(a)
    mplot.append(m)
    #Shuffle Input Set
    DesiredOutputSet = np.delete(df_shuffle, np.s_[:inputsize], axis=1)  
    InputSet = np.insert(np.delete(df_shuffle, np.s_[inputsize:], axis=1), 0, bias, axis=1)
    InputBatch = [InputSet[i:i + batchsize] for i in range(0, len(InputSet), batchsize)]
    DesiredOutputBatch = [DesiredOutputSet[i:i + batchsize] for i in range(0, len(DesiredOutputSet), batchsize)]
    
    for i in range(len(DesiredOutputBatch)):
        
        Input = InputBatch[i]
        d = DesiredOutputBatch[i]
       
        v,y = forward(Input,2,w,af)
        e = error(d,y['y_out'])
        squareerror = 1/(2*len(d[0]))*np.sum(np.square(e))
        
        delta = gradient(H,e,af,y,w)
        dw = delta_weights(H, Input, a, delta, y, batchsize)
        w = new_weights(H, m, w, dw_0, dw)
        dw_0 = dw
        y_output = np.argmax(y['y_out'], axis=0)+1
        final_output = np.append(final_output, y_output)
        sumerror.append(squareerror)
        
    epocherror.append(np.average(sumerror))
    
    if (k+1) %5 == 0: 
        InputValidationBatch = [InputValidationSet[i:i + batchsize] for i in range(0, len(InputValidationSet), batchsize)]
        DesiredValidationOutputBatch = [DesiredOutputValidationSet[i:i + batchsize] for i in range(0, len(DesiredOutputValidationSet), batchsize)]
        y_predicted = []
        val_e = []
        for i in range(len(DesiredValidationOutputBatch)):
            Input = InputValidationBatch[i]
            d1 = DesiredValidationOutputBatch[i]
            outputs = forward(Input,2,w,af)[1]
            validation_e = error(d1,outputs['y_out'])
            PredictedOutput = np.argmax(outputs['y_out'], axis=0)+1
            val_squareerror = 1/(2*batchsize)*np.sum(np.square(validation_e))
            
            val_e.append(val_squareerror)
            y_predicted = np.append(y_predicted,PredictedOutput )
        validation_error.append(np.average(val_e))
        
        classes = len(DesiredOutputSet[0]) #find number of classes

        cm = [[sum([((OutputValidationSet)[i] == true_class) and (y_predicted[i] == pred_class) 
                        for i in range(len(OutputValidationSet))])
                   for pred_class in range(1, classes + 1)] 
                   for true_class in range(1, classes + 1)]
        cm = np.array(cm,dtype=float)

        Total_C = np.sum(cm,axis=0)
        Total_R = np.sum(cm,axis=1)
        TP = np.diagonal(cm)
        FP = Total_C-TP
        TN = [sum(Total_C)-Total_C[i]-Total_R[i]+cm[i][i] for i in range(classes)]
        FN = Total_R - TP
        Recall = TP/(TP+FN)
        Precision = TP/(TP+FP)
        Accuracy = (TP+TN)/(TP+FP+TN+FN)
        F1Scores = 2*Precision*Recall/(Precision+Recall)
        MCC = (TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
        misclassification = np.append(misclassification,1-Accuracy)
        
   # if (k+1) %10 == 0:
        a = ainit*1/(1.001+k)
        m = minit*(1-k/epoch)/((1-minit)+minit*(1-k/epoch))
        
    print("Per Epoch:--- %s seconds ---" % (time.time() - epochstart_time))
print("Total Training Time:--- %s seconds ---" % (time.time() - start_time))

plt.plot(np.arange(len(epocherror)),epocherror)
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.title('Training Error')
plt.show()
plt.plot(np.arange(0,epoch,5),validation_error)
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.title('Validation Error')
plt.show()
plt.plot(np.arange(len(epocherror)),lt)
plt.xlabel("Epochs")
plt.ylabel("Learning Rate")
plt.title('Learning Rate Per Epoch')
plt.show()
plt.plot(np.arange(len(epocherror)),mplot)
plt.xlabel("Epochs")
plt.ylabel("Momentum")
plt.title('Momentum Per Epoch')
plt.show()

np.savetxt(Network + "_training_sumofsquares.csv", np.stack([np.arange(len(epocherror)),epocherror],axis=1), delimiter=",")
np.savetxt(Network + "_validation_sumofsquares.csv", np.stack([np.arange(0,epoch,5),validation_error],axis=1), delimiter=",")
np.savetxt(Network + "_validation_misclassification.csv", misclassification, delimiter=",")
np.savetxt(Network + "_confusion_matrix.csv", cm, delimiter=",")
np.savetxt(Network + "_Recall.csv", Recall, delimiter=",")
np.savetxt(Network + "_Precision.csv", Precision, delimiter=",")
np.savetxt(Network + "_Accuracy.csv", Accuracy, delimiter=",")
np.savetxt(Network + "_F1_Scores.csv", F1Scores, delimiter=",")
np.savetxt(Network + "_Matthews_Correlation_Coeff.csv", MCC, delimiter=",")

with open(Network + "_trained_weights.csv", 'w') as csv_file:  
    
    for i in range(H):
        l = str(i+1)
        np.savetxt(csv_file, w['w_h'+l],delimiter=',',header='w_h'+l,fmt='%s')
    np.savetxt(csv_file, w['w_out'],delimiter=',',header='w_out',fmt='%s')
    


In [ ]:
#Test Dataset Predictions

H=2
bias=1
NetworkA = "Network" + str(input('Network: '))
af_a = activation_function(H)
NetworkB = "Network" + str(input('Network: '))
af_b = activation_function(H)
with open("NetworkA_trained_weights.csv", 'r') as file:
    NetworkA_w={}
    my_reader = csv.reader(file, delimiter=',')

    for row in my_reader:
        if len(row)==1:
           l = row[0].split()[1]
           NetworkA_w[l]=[]
        else:
            row = (list(map(float, row)))
            NetworkA_w[l].append(row)


with open("NetworkB_trained_weights.csv", 'r') as file:
    NetworkB_w={}
    my_reader = csv.reader(file, delimiter=',')

    for row in my_reader:
        if len(row)==1:
           l = row[0].split()[1]
           NetworkB_w[l]=[]
        else:
            row = (list(map(float, row)))
            NetworkB_w[l].append(row)

for i in range(H):
    l = str(i+1) 
    NetworkA_w['w_h'+l] = np.array(NetworkA_w['w_h'+l])
    NetworkB_w['w_h'+l] = np.array(NetworkB_w['w_h'+l])
#Output Layer   
NetworkA_w['w_out'] = np.array(NetworkA_w['w_out'])
NetworkB_w['w_out'] = np.array(NetworkB_w['w_out'])

filedata = 'C:/Users/KEC/OneDrive - University of the Philippines/MSEE/CS280/PA2/CS280_PA2_Data/CS280_PA2_Data/test_set.csv'

with open(filedata, 'r') as file:
    testset = np.array(list(csv.reader(file, delimiter=',',quoting=csv.QUOTE_NONNUMERIC)))

testset = np.insert(testset, 0, bias, axis=1)

testbatch = [testset[i:i + batchsize] for i in range(0, len(testset), batchsize)]

NetworkA_Predicted = []
NetworkB_Predicted = []
for i in range(len(testbatch)):
     Input1 = testbatch[i]
     
     NetworkA_output = forward(Input1,2,NetworkA_w,af_a)[1]
     PredictedA = np.argmax(NetworkA_output['y_out'], axis=0)+1
     NetworkA_Predicted = np.append(NetworkA_Predicted,PredictedA )    
    
     NetworkB_output = forward(Input1,2,NetworkB_w,af_b)[1]
     PredictedB = np.argmax(NetworkB_output['y_out'], axis=0)+1
     NetworkB_Predicted = np.append(NetworkB_Predicted,PredictedB )   

np.savetxt("predictions_for_test_tanh.csv", NetworkA_Predicted, delimiter=",")
np.savetxt("predictions_for_test_leakyrelu.csv", NetworkB_Predicted, delimiter=",")
        